In [1]:
import matplotlib.pyplot as plt
import numpy as np
import io,os,glob,scipy
from h5py import File
from netCDF4 import Dataset
import datetime as dt
import lpdm

In [4]:
with open('/Users/seancrowell/Google Drive/My Drive/CarbonFOX/simulations/mair/coarse_col_receptors.asc') as f:
    files = f.read().splitlines()
ft = np.zeros((776,24,100,133))
for fi in files:
    ft += Dataset(fi)['foot'][:]

### Simplest setup:
Prior = ACES

Truth = 2 x ACES

Obs Uncert = 0.01ppm

Prior Uncert = abs(ACES)

Transport is perfect

In [5]:
shat = Dataset('shat_aces2xtruth_0.01ppm.nc4')['shat'][:]
apri = Dataset('sprior.nc')['sprior'][:]
true = Dataset('strue.nc')['sprior'][:]
sig = Dataset('prior_uncert.nc')['sigma'][:]


FileNotFoundError: [Errno 2] No such file or directory: 'strue.nc'

In [ ]:

fig,axs = plt.subplots(1,3,figsize=(14,4))
#prior
g = axs[0].pcolormesh(apri[12:36].mean(0),cmap=plt.cm.Reds,vmin=0,vmax=20); 
plt.colorbar(g,ax=axs[0])
axs[0].set_title('Prior Emission')
#Posterior
g = axs[1].pcolormesh(shat[12:36].mean(0),cmap=plt.cm.Reds,vmin=0,vmax=20); 
plt.colorbar(g,ax=axs[1])
axs[1].set_title('Posterior Emission')
#True
g = axs[2].pcolormesh(true[12:36].mean(0),cmap=plt.cm.Reds,vmin=0,vmax=20); 
plt.colorbar(g,ax=axs[2])
axs[2].set_title('True Emission')

In [3]:
inds = np.where(true.mean(0).flatten() > 0.1)
print(f'Mean Post Flux Error: {np.mean((shat-true).mean(0).flatten()[inds]):5.3f}')
print(f'Mean Prior Flux Error: {np.mean((apri-true).mean(0).flatten()[inds]):5.3f}')
print(f'Mean Flux Increment: {np.mean((apri-shat).mean(0).flatten()[inds]):5.3f}')
print()
print(f'Median Post Flux Error: {np.median((shat-true).mean(0).flatten()[inds]):5.3f}')
print(f'Median Prior Flux Error: {np.median((apri-true).mean(0).flatten()[inds]):5.3f}')
print(f'Median Flux Increment: {np.median((apri-shat).mean(0).flatten()[inds]):5.3f}')
print()
print(f'Post Flux RMSE: {np.sqrt(np.mean(((shat-true).mean(0).flatten()**2)[inds])):5.3f}')
print(f'Prior Flux RMSE: {np.sqrt(np.mean(((apri-true).mean(0).flatten()**2)[inds])):5.3f}')
print(f'Prior-Post RMSE: {np.sqrt(np.mean(((apri-shat).mean(0).flatten()**2)[inds])):5.3f}')

NameError: name 'true' is not defined

In [ ]:
inds = np.where(true.mean(0).flatten() > 0.1)
fig,axs = plt.subplots(1,1)
axs.hist((shat-true).mean(0).flatten()[inds]/(apri-true).mean(0).flatten()[inds]),bins=np.linspace(0,2,201));

In [ ]:
fig,axs = plt.subplots(2,2,figsize=(10,10))

ax = axs[0,0]
g = ax.pcolormesh(apri[-5:-4].mean(0)/true[-5:-4].mean(0),vmin=0.5,vmax=1.5)
plt.colorbar(g,ax=ax)
ax.set_title('Prior:Truth Ratio')

ax = axs[0,1]
g = ax.pcolormesh(shat[-5:-4].mean(0)/true[-5:-4].mean(0),vmin=0.5,vmax=1.5)
plt.colorbar(g,ax=ax)
ax.set_title('Posterior:Truth Ratio')

ax = axs[1,0]
g = ax.pcolormesh(sig[12],vmin=0.1,vmax=20)
plt.colorbar(g,ax=ax)
ax.set_title('Sigma')

ax = axs[1,1]
g = ax.pcolormesh(ft.sum((0,1)))
plt.colorbar(g,ax=ax)
ax.set_title('Footprint Total Sensitivity')

In [ ]:
ctl = lpdm.lpdm('config.ini')
apri_v = ctl.load_file('sprior.nc','sprior')
tru_v = ctl.load_file('aces2x_4km_6km_prior_daily_mean.nc','sprior')

hsp = ctl.convolve(apri_v)
hst = ctl.convolve(tru_v)
hsp2 = np.loadtxt('hsprior.txt')
zhsp = np.loadtxt('zhsp.txt')
hsh = np.loadtxt('hshat.txt')

plt.hist(hsp-hst,bins=np.linspace(-20,20,401),histtype='step');
plt.hist(hsh-hst,bins=np.linspace(-20,20,401),histtype='step');
plt.legend(['Prior MDM','Posterior MDM'])
plt.title('Mismatch with Obs Before and After Optimization')

In [ ]:
apri.shape

In [ ]:
ctl = lpdm.lpdm('config.ini')

## Comparison of Different Obs Errors

In [ ]:
shat_001 = Dataset('shat_aces1.3xtruth_0.01ppm.nc')['shat'][:]
shat_05 = Dataset('shat_aces1.3xtruth_0.5ppm.nc')['shat'][:]
shat_2 = Dataset('shat_aces1.3xtruth_2.0ppm.nc')['shat'][:]

shat_001_v = Dataset('shat_vulcantruth_0.01ppm.nc4')['shat'][:]
shat_05_v = Dataset('shat_vulcantruth_0.5ppm.nc4')['shat'][:]
shat_2_v = Dataset('shat_vulcantruth_2.0ppm.nc4')['shat'][:]

apri = Dataset('sprior.nc')['sprior'][:]
aces_true = Dataset('aces1.3x_4km_6km_prior_daily_mean.nc')['sprior'][:]
vulcan_true = Dataset('vulcan_4km_6km_prior_daily_mean.nc')['sprior'][:]
sig = Dataset('prior_uncert.nc')['sigma'][:]

ctl = lpdm.lpdm('config.ini')
apri_v = ctl.load_file('sprior.nc','sprior')
aces_tru_v = ctl.load_file('aces1.3x_4km_6km_prior_daily_mean.nc','sprior')
vulcan_tru_v = ctl.load_file('vulcan_4km_6km_prior_daily_mean.nc','sprior')

In [ ]:
plt.plot(np.arange(0,48,3),apri[:,20:50,75:100].mean((1,2)))
plt.plot(np.arange(0,48,3),aces_true[:,20:50,75:100].mean((1,2)),'tab:orange')
plt.plot(np.arange(0,48,3),vulcan_true[:,20:50,75:100].mean((1,2)),'tab:green')
plt.plot(np.arange(0,48,3),shat_001[:,20:50,75:100].mean((1,2)),'tab:orange',marker='*')
plt.plot(np.arange(0,48,3),shat_05[:,20:50,75:100].mean((1,2)),'tab:orange',ls='--')
plt.plot(np.arange(0,48,3),shat_2[:,20:50,75:100].mean((1,2)),'tab:orange',ls='-.')

plt.plot(np.arange(0,48,3),shat_001_v[:,20:50,75:100].mean((1,2)),'tab:green',marker='*')
plt.plot(np.arange(0,48,3),shat_05_v[:,20:50,75:100].mean((1,2)),'tab:green',ls='--')
plt.plot(np.arange(0,48,3),shat_2_v[:,20:50,75:100].mean((1,2)),'tab:green',ls='-.')

plt.legend(['Prior','ACES 1.3X','Vulcan','Post (A13 0.01ppm)','Post (A13 0.5ppm)','Post (A13 2.0ppm)','Post (V 0.01ppm)','Post (V 0.5ppm)','Post (V 2.0ppm)'])
plt.title('NYC Core Hourly Mean Emissions');
plt.xticks([0,12,24,36,48],['2023-09-02 OZ','2023-09-02 12Z','2023-09-03 0Z','2023-09-03 12Z','2023-09-04 0Z'],rotation=45);
#plt.xticklabels()

In [ ]:


fig,axs = plt.subplots(3,2,figsize=(10,14))

ax = axs[0,0]
g = ax.pcolormesh((apri[7:-1].sum(0)-aces_true[7:-1].sum(0))/aces_true[7:-1].sum(0),vmin=-0.5,vmax=0.)
plt.colorbar(g,ax=ax)
ax.set_title('Prior:Truth (ACES1.3x) Ratio')

ax = axs[0,1]
g = ax.pcolormesh((shat_001[7:-1].sum(0)-aces_true[7:-1].sum(0))/aces_true[7:-1].sum(0),vmin=-0.5,vmax=0.)
plt.colorbar(g,ax=ax)
ax.set_title('Posterior:Truth (ACES1.3x) Ratio')

ax = axs[1,0]
g = ax.pcolormesh((apri[7:-1].mean(0)-vulcan_true[7:-1].mean(0)),vmin=-20,vmax=20)
plt.colorbar(g,ax=ax)
ax.set_title('Prior:Truth (Vulcan)')

ax = axs[1,1]
g = ax.pcolormesh((shat_001_v[7:-1].mean(0)-vulcan_true[7:-1].mean(0)),vmin=-20,vmax=20)
plt.colorbar(g,ax=ax)
ax.set_title('Posterior:Truth (Vulcan)')

ax = axs[2,0]
g = ax.pcolormesh(sig[12],vmin=0.1,vmax=20)
plt.colorbar(g,ax=ax)
ax.set_title('Sigma')

ax = axs[2,1]
g = ax.pcolormesh(ft.sum((0,1)))
plt.colorbar(g,ax=ax)
ax.set_title('Footprint Total Sensitivity')

In [ ]:
hsp = ctl.convolve(apri_v)
hst_aces2 = ctl.convolve(aces_tru_v)
hst_vulcan = ctl.convolve(vulcan_tru_v)
zhsp = np.loadtxt('zhsp.txt')
hsh_001 = np.loadtxt('hshat_aces1.3x_0.01ppm.txt')
hsh_05 = np.loadtxt('hshat_aces1.3x_0.5ppm.txt')
hsh_2 = np.loadtxt('hshat_aces1.3x_0.01ppm.txt')
hsh_001_v = np.loadtxt('hshat_vulcan_0.01ppm.txt')
hsh_05_v = np.loadtxt('hshat_vulcan_0.5ppm.txt')
hsh_2_v = np.loadtxt('hshat_vulcan_2.0ppm.txt')

plt.hist(hsp-hst_aces2,bins=np.linspace(-20,20,401),histtype='step',color='tab:blue');
plt.hist(hsp-hst_vulcan,bins=np.linspace(-20,20,401),histtype='step',color='tab:orange');
plt.legend(['ACES13 Prior MDM','Vulcan Prior MDM'])
plt.title('Mismatch with Obs Before and After Optimization')


In [ ]:
plt.figure()
plt.hist(hsh_001-hst_aces2,bins=np.linspace(-20,20,401),histtype='step',color='tab:blue');
plt.hist(hsh_001_v-hst_vulcan,bins=np.linspace(-20,20,401),histtype='step',color='tab:orange');
plt.legend(['ACES13 Posterior MDM','Vulcan Posterior MDM'])
plt.title('Mismatch with Obs Before and After Optimization')